In [14]:
from gqe.operator_pool.uccsd import UCCSD, generate_molecule
# distances = [0.5, 0.6, 0.7, 0.7414, 0.8, 0.9, 1.0, 1.5, 2.0]
distance = 0.5
transformation = 'jordan-wigner'
is_bravyi = transformation == 'bravyi-kitaev'
molecule = generate_molecule("H", "H", distance, "sto-3g", bravyi_kitaev=is_bravyi)

converged SCF energy = -1.0429962745401


In [1]:
from transformers import GPT2Config, GPT2LMHeadModel
GPT2Config().to_dict()

{'vocab_size': 50257,
 'n_positions': 1024,
 'n_embd': 768,
 'n_layer': 12,
 'n_head': 12,
 'n_inner': None,
 'activation_function': 'gelu_new',
 'resid_pdrop': 0.1,
 'embd_pdrop': 0.1,
 'attn_pdrop': 0.1,
 'layer_norm_epsilon': 1e-05,
 'initializer_range': 0.02,
 'summary_type': 'cls_index',
 'summary_use_proj': True,
 'summary_activation': None,
 'summary_first_dropout': 0.1,
 'summary_proj_to_labels': True,
 'scale_attn_weights': True,
 'use_cache': True,
 'scale_attn_by_inverse_layer_idx': False,
 'reorder_and_upcast_attn': False,
 'bos_token_id': 50256,
 'eos_token_id': 50256,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': None,
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_lengt

In [15]:
from benchmark.molecule import DiatomicMolecularHamiltonian
from qwrapper.hamiltonian import compute_ground_state
nqubit = 4
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=is_bravyi)
ge = compute_ground_state(hamiltonian)
print("ground state:", ge)


Starting to parse FermionOperator using 4 qubits...

Operator t:  0.3798313517809532 [] +
-0.04221755692243385 [X0 X1 Y2 Y3] +
0.04221755692243385 [X0 Y1 Y2 X3] +
0.04221755692243385 [Y0 X1 X2 Y3] +
-0.04221755692243385 [Y0 Y1 X2 X3] +
0.21393531024521326 [Z0] +
0.17992650976405974 [Z0 Z1] +
0.13459240346368836 [Z0 Z2] +
0.17680996038612218 [Z0 Z3] +
0.21393531024521334 [Z1] +
0.17680996038612218 [Z1 Z2] +
0.13459240346368836 [Z1 Z3] +
-0.3691443152437658 [Z2] +
0.18620984259247125 [Z2 Z3] +
-0.3691443152437658 [Z3]
Term, coeff:  () 0.3798313517809532
Term, coeff:  ((0, 'Z'),) 0.21393531024521326
Index, p_char:  0 Z
Term, coeff:  ((1, 'Z'),) 0.21393531024521334
Index, p_char:  1 Z
Term, coeff:  ((2, 'Z'),) -0.3691443152437658
Index, p_char:  2 Z
Term, coeff:  ((3, 'Z'),) -0.3691443152437658
Index, p_char:  3 Z
Term, coeff:  ((0, 'Z'), (1, 'Z')) 0.17992650976405974
Index, p_char:  0 Z
Index, p_char:  1 Z
Term, coeff:  ((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')) 0.04221755692243385
Index, p

In [16]:
from qwrapper.obs import PauliObservable
from gqe.common.initializer import HFStateInitializer
from qswift.compiler import DefaultOperatorPool
from gqe.mingpt.cost import EnergyCost

# prepare operator_pool
uccsd = UCCSD(4, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIII"))
initializer = HFStateInitializer(n_electrons=2)
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(4, [], "qulacs")))

pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                    [0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025, 0.05, -0.05, 0.1, -0.1])


hf state: -1.0429962745400951


In [18]:
from transformers.models.gpt2.modeling_gpt2 import *

In [ ]:
from transformers import GPT2Config
class Transformer(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        gpt2config = cfg
        self.transformer = GPT2LMHeadModel(gpt2config)        
    
    def generate_logits(self, idx):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0)  # shape (1, t)

        # forward the GPT model itself
        logits = self.transformer(idx)[0]
        return logits

    def cost(self, idx):
        idx_output, logits_tensor = self.generate(idx, self.n_gates)
        energies = self._cost.energy(idx_output)
        mean_logits = torch.mean(logits_tensor, 1)
        print("mean_logits", mean_logits * self.energy_scaling)
        print("energies:", energies)
        print("mean:", torch.mean(energies))
        loss = torch.nn.MSELoss()
        return loss(torch.exp(-mean_logits), torch.exp(-energies / self.energy_scaling))
        # return loss(mean_logits, energies)

    def generate(self, idx, max_new_tokens):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        b_size = idx.size(dim=0)
        condition_length = idx.size(dim=1)
        # logits_tensor = torch.empty((max_new_tokens, b_size))
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits_base = self.generate_logits(idx_cond)
            logits = logits_base[:, -1, :]
            probs = F.softmax(-self.temperature * logits, dim=-1)
            # either sample from the distribution or take the most likely element
            idx_next = torch.multinomial(probs, num_samples=1)
            # logits_tensor[pos] = torch.gather(logits, 1, idx_next).flatten()
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)
        idx = idx[:, condition_length:]
        # print(logits_tensor.T)
        return idx, torch.gather(logits_base, 2, idx.reshape(b_size, -1, 1)).reshape(b_size, -1)

    def forward(self, idx):
        loss = self.cost(idx)
        return loss

In [ ]:
import lightning as L

fabric = L.Fabric()
fabric.launch()

model = Transformer()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model, optimizer = fabric.setup(model, optimizer)

model.train()
for epoch in range(20):
    optimizer.zero_grad()
    output = model(input, target)
    loss = torch.nn.functional.nll_loss(output, target.view(-1))
    fabric.backward(loss)
    optimizer.step()

In [9]:
from ml_collections import config_dict, ConfigDict

# placeholder = config_dict.FieldReference(0)
cfg = ConfigDict()
# setup parameters
cfg.sparams = sparams = ConfigDict()
sparams.nqubit = 4
sparams.ngates = config_dict.FieldReference(0)
sparams.seed = 3047
sparams.transformation = 'jordan-wigner'
sparams.distances = [0.5, 0.6, 0.7, 0.7414, 0.8, 0.9, 1.0, 1.5, 2.0]  # choices of the distance between two atoms
# hyperparameters
cfg.hparams = hparams = ConfigDict()
hparams.iter = 100
cfg.gptparams = gptparams = ConfigDict()
gptparams.ngates = sparams.ngates

In [13]:
cfg.sparams.ngates = 40
cfg.gptparams.ngates
cfg.sparams.ngates

40

In [25]:
from ml_collections import config_dict

placeholder = config_dict.FieldReference(0)
cfg = config_dict.ConfigDict()
cfg.placeholder = placeholder
cfg.optional = config_dict.placeholder(int)
cfg.nested = config_dict.ConfigDict()
cfg.nested.placeholder = placeholder

try:
  cfg.optional = 'tom'  # Raises Type error as this field is an integer.
except TypeError as e:
  print(e)

cfg.optional = 1555  # Works fine.
cfg.placeholder = 1  # Changes the value of both placeholder and
                     # nested.placeholder fields.

print(cfg)

Could not override field 'optional' (reference). tom is of type <class 'str'> but should be of type <class 'int'>
nested:
  placeholder: 1
optional: 1555
placeholder: 1



In [34]:
cfg.nested.placeholder

23

In [29]:
cfg

nested:
  placeholder: &id002 !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: &id001 !!python/name:builtins.int ''
    _ops: []
    _required: false
    _value: 1
optional: !!python/object:ml_collections.config_dict.config_dict.FieldReference
  _field_type: *id001
  _ops: []
  _required: false
  _value: 1555
placeholder: *id002

In [6]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Get the first 100 characters of Lorem Ipsum
lorem_ipsum_100 = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut."

# Create an image
width, height = 600, 60
image = Image.new('RGB', (width, height), color = 'white')
draw = ImageDraw.Draw(image)

# Use a truetype font (This uses the default font. For a different font, provide the path)
font = ImageFont.truetype("Arial",15)

# Draw the text onto the image
draw.text((10, 10), lorem_ipsum_100, font=font, fill='black')

# Save the image
image.save('lorem_ipsum_100.png')


In [3]:
import torch
import pytorch_lightning as pl
from torchvision import datasets, transforms
from torch.nn import functional as F
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from tqdm import tqdm
from itertools import cycle

import pytorch_lightning as pl
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from tqdm import tqdm
from itertools import cycle

class RollingLyricsProgressBar(pl.callbacks.ProgressBar):

    def __init__(self, width=30):
        super(RollingLyricsProgressBar, self).__init__()
        
        # Define lyrics and join them for a continuous flow
        self.lyrics = [
            "We're no strangers to love",
            "You know the rules and so do I",
            # ... Add the rest of the lyrics.
            "Never gonna give you up",
            "Never gonna let you down",
        ]
        self.full_lyrics = ' | '.join(self.lyrics) + ' | '
        
        self.width = width
        self.position = 0
        
        # Create an image of the lyrics
        fnt = ImageFont.load_default()
        dctx = ImageDraw.Draw(Image.new('1', (1, 1)))
        text_width, text_height = dctx.textsize(self.full_lyrics, font=fnt)
        self.image = Image.new('1', (text_width, text_height), color=0)
        dctx = ImageDraw.Draw(self.image)
        dctx.text((0, 0), self.full_lyrics, font=fnt, fill=1)

        
    def init_sanity_tqdm(self):
        bar = tqdm(position=self.position, leave=True)
        bar.set_postfix_str(self.get_next_pixels())
        return bar

    def init_train_tqdm(self):
        bar = tqdm(position=self.position, leave=True)
        bar.set_postfix_str(self.get_next_pixels())
        return bar

    def get_next_pixels(self):
        # Extract a portion of the image to represent current progress bar state
        segment = self.image.crop((self.position, 0, self.position+self.width, self.image.height))
        np_segment = np.asarray(segment).astype(int)
        
        # Convert binary image to a string of pixel representation (e.g., using `█` for filled pixel)
        bar_str = ''
        for row in np_segment:
            if np.any(row):  # if any pixel in row is ON
                bar_str += '█'
            else:
                bar_str += ' '

        self.position = (self.position + 1) % (self.image.width - self.width)

        return bar_str

# Example of using the progress bar
trainer = pl.Trainer(callbacks=[RollingLyricsProgressBar()], max_epochs=100)


class SimpleCNN(pl.LightningModule):

    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(8 * 8 * 64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 8 * 8 * 64)
        x = self.fc1(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

    def train_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        dataset = datasets.CIFAR10('~/.cache/', train=True, download=True, transform=transform)
        loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
        return loader

def main():
    model = SimpleCNN()
    trainer = pl.Trainer(callbacks=[RollingLyricsProgressBar(width=50)], max_epochs=3, progress_bar_refresh_rate=1)
    trainer.fit(model)


main()


AttributeError: 'ImageDraw' object has no attribute 'textsize'